# Detecção de Bots Dataset Telegram

Assumimos que bots apresentam comportamento coordenado com outros usuários ou bots. Utilizando hipóteses de comportamento humano, serão criados grafos com candidatos a bots que serão filtrados usando algoritmos de comunidade. As hipóteses definidas foram:

1. Pessoas dificilmente compartilham mensagens poucos segundos após ser enviada
2. Pessoas dificilmente compartilham mensagens no mesmo horário frequentemente
3. Pessoas apresentam intervalos de tempo entre mensagens caóticos. Bots tendem a enviar mensagens de forma mais sistematizada

In [ ]:
# dependências
import networkx as nx
from networkx.algorithms import community
import concurrent.futures
import os

import warnings
import os
os.chdir("..")
from util.load_graph import load_graph_by_edge, get_driver
from util.bot_filtering import rank_bot_suspicion

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
driver = get_driver()

## Hipótese 1: Usuários que compartilham mensagens rápidos demais

![rapid share](../../images/rapid_share.png)

In [ ]:
G_rapid = load_graph_by_edge(driver, 'RAPID_SHARE')

rapid_bots = rank_bot_suspicion(G_rapid) 
rapid_bots

In [ ]:
rapid_bots.describe()

In [ ]:
filtered_rapid_bots = rapid_bots[
    rapid_bots['bot_suspicion_score'] >
    rapid_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_rapid_bots

## Hipótese 2: Usuários que compartilham mensagens no mesmo horário frequentemente

![hourly](../../images/hourly.png)

In [ ]:
G_hourly = load_graph_by_edge(driver, 'HOURLY_SHARED')

hourly_bots = rank_bot_suspicion(G_hourly) 
hourly_bots

In [ ]:
hourly_bots.describe()

In [ ]:
filtered_hourly_bots = hourly_bots[
    hourly_bots['bot_suspicion_score'] >
    hourly_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_hourly_bots

## Hipótese 3: Usuários que compartilham mensagens com pouca variação
![hourly](../../images/metronome.png)

In [ ]:
G_metronome = load_graph_by_edge(driver, 'METRONOME_SIMILAR')

metronome_bots = rank_bot_suspicion(G_metronome) 
metronome_bots

In [ ]:
metronome_bots.describe()

In [ ]:
filtered_metronome_bots = metronome_bots[
    metronome_bots['bot_suspicion_score'] >
    metronome_bots['bot_suspicion_score'].quantile(0.8)
]
filtered_metronome_bots

## Avaliando Semelhança entre Técnicas de Detecção

In [ ]:
metronome_susp = set(filtered_metronome_bots['user_id'])
hourly_susp = set(filtered_hourly_bots['user_id'])
rapid_susp = set(filtered_rapid_bots['user_id'])

In [ ]:
overlap_mh = metronome_susp & hourly_susp
overlap_mr = metronome_susp & rapid_susp
overlap_hr = hourly_susp & rapid_susp

len(overlap_mh), len(overlap_mr), len(overlap_hr)

In [ ]:
def jaccard(a, b):
    return len(a & b) / len(a | b) if (a | b) else 0

j12 = jaccard(metronome_susp, hourly_susp)
j13 = jaccard(metronome_susp, rapid_susp)
j23 = jaccard(hourly_susp, rapid_susp)

j12, j13, j23

In [ ]:
def overlap_ratio(a, b):
    return len(a & b) / min(len(a), len(b)) if min(len(a), len(b)) else 0

o12 = overlap_ratio(metronome_susp, hourly_susp)
o13 = overlap_ratio(metronome_susp, rapid_susp)
o23 = overlap_ratio(rapid_susp, hourly_susp)

o12, o13, o23

In [ ]:
highest_confident = metronome_susp & hourly_susp & rapid_susp
len(highest_confident)

In [ ]:
concatenated = metronome_susp | hourly_susp | rapid_susp
len(concatenated)

In [ ]:
import pandas as pd

pd.DataFrame({'user_id': list(highest_confident)}).to_csv(
    '../data/highest_confident_bots.csv', index=False
)

pd.DataFrame({'user_id': list(concatenated)}).to_csv(
    '../data/all_suspected_bots.csv', index=False
)